En el análisis realizado, concluimos la siguiente información:
- Tenemos como entrada (x) 200.000 datos de transacciones (big data).
- No hay transacciones duplicadas ni inválidas.
- Tenemos como salida (y), una clasificación binaria (0,1): fraude o no (columna Is_fraud).
- Tenemos un total de 10088 transacciones etiquetadas comno fraudes.
- Al estar utilizando un dataset ya existente, el etiquetado realizado es correcto y no necesita de un trabajo de etiquetado adicional.
- Tenemos diferentes tipos de transacciones: crédito, débito, facturas, transferencia, y retiradas.

Una vez realizado este análisis, comenzamos con el preprocesamiento de los datos, con el objetivo de establecer un "Data Pipeline". A continuación, se detalla el Data Pipeline definido para preparar los datos antes de comenzar con el modelado ML.


# 1. Limpieza de datos
Ya hemos confirmado que no tenemos valores nulos ni duplicados, por lo que procedemos con la transformación de datos.

Vamos a eliminar columnas irrelevantes, es decir, columnas que no aportan información útil:
- Custome_ID, Customer_Name, Customer_Contact, Customer_Email, Transaction_ID, Merchant_ID. Estos son identificadores no predictivos.
- Transaction_Description. Texto difícil de utilizar sin procesamiento NLP.
- Transaction_Location. Tenemos ya almacenada información del lugar.

In [38]:
import pandas as pd

In [59]:
df = pd.read_csv("data/raw_data.csv")

In [60]:
print(df.columns)


Index(['Customer_ID', 'Customer_Name', 'Gender', 'Age', 'State', 'City',
       'Bank_Branch', 'Account_Type', 'Transaction_ID', 'Transaction_Date',
       'Transaction_Time', 'Transaction_Amount', 'Merchant_ID',
       'Transaction_Type', 'Merchant_Category', 'Account_Balance',
       'Transaction_Device', 'Transaction_Location', 'Device_Type', 'Is_Fraud',
       'Transaction_Currency', 'Customer_Contact', 'Transaction_Description',
       'Customer_Email'],
      dtype='object')


In [61]:
drop_columns = ["Customer_ID", "Customer_Name", "Customer_Contact", "Customer_Email", "Transaction_ID", "Merchant_ID","Transaction_Description", "Transaction_Location"]
df.drop(columns=drop_columns, inplace=True)
print(df.columns)

Index(['Gender', 'Age', 'State', 'City', 'Bank_Branch', 'Account_Type',
       'Transaction_Date', 'Transaction_Time', 'Transaction_Amount',
       'Transaction_Type', 'Merchant_Category', 'Account_Balance',
       'Transaction_Device', 'Device_Type', 'Is_Fraud',
       'Transaction_Currency'],
      dtype='object')


# 2. Codificación de variables categóricas
Vamos a convertir columnas categóricas a valores numéricos, ya que generalmente los algoritmos de ML trabajan mejor con valores numéricos. Vamos a aplicar los siguientes métodos:
- one-hot encoding: vamos a crear una nueva columna para cada categoría y usar valores 0 y 1. Esto lo haremos en aquellas variables que tienen pocas categorías: Gender, Account_Type, Transaction_Type, Merchant_Category, Device_Type, Transaction_Currency
- Label encoding: vamos a convertir cada categoría en un número entero. Esto lo aplicaremos en variables con muchas categorías: City, State, Bank_Branch

In [ ]:
df = pd.get_dummies(df, columns=["Gender", "Account_Type","Transaction_Type", "Merchant_Category", "Device_Type", "Transaction_Currency"], drop_first=True)
print(df.columns)

Index(['Age', 'State', 'City', 'Bank_Branch', 'Transaction_Date',
       'Transaction_Time', 'Transaction_Amount', 'Account_Balance', 'Is_Fraud',
       'Gender_Male', 'Account_Type_Checking', 'Account_Type_Savings',
       'Transaction_Device_ATM Booth Kiosk', 'Transaction_Device_Bank Branch',
       'Transaction_Device_Banking Chatbot',
       'Transaction_Device_Biometric Scanner',
       'Transaction_Device_Debit/Credit Card',
       'Transaction_Device_Desktop/Laptop', 'Transaction_Device_Mobile Device',
       'Transaction_Device_POS Mobile App',
       'Transaction_Device_POS Mobile Device',
       'Transaction_Device_POS Terminal',
       'Transaction_Device_Payment Gateway Device',
       'Transaction_Device_QR Code Scanner',
       'Transaction_Device_Self-service Banking Machine',
       'Transaction_Device_Smart Card', 'Transaction_Device_Tablet',
       'Transaction_Device_Virtual Card', 'Transaction_Device_Voice Assistant',
       'Transaction_Device_Wearable Device', 'Tr

In [ ]:
from sklearn.preprocessing import LabelEncoder

for col in ["City", "State", "Bank_Branch", "Transaction_Device"]:
    df[col] = LabelEncoder().fit_transform(df[col])

In [57]:
df.head()

,Age,State,City,Bank_Branch,Transaction_Date,Transaction_Time,Transaction_Amount,Account_Balance,Transaction_Device,Transaction_Location,...,Transaction_Type_Transfer,Transaction_Type_Withdrawal,Merchant_Category_Electronics,Merchant_Category_Entertainment,Merchant_Category_Groceries,Merchant_Category_Health,Merchant_Category_Restaurant,Device_Type_Desktop,Device_Type_Mobile,Device_Type_POS
0,60,15,127,127,23-01-2025,16:04:07,32415.45,74557.27,Voice Assistant,"Thiruvananthapuram, Kerala",...,True,False,False,False,False,False,True,False,False,True
1,51,18,100,100,11-01-2025,17:14:53,43622.60,74622.66,POS Mobile Device,"Nashik, Maharashtra",...,False,False,False,False,False,False,True,True,False,False
2,20,4,13,13,25-01-2025,03:09:52,63062.56,66817.99,ATM,"Bhagalpur, Bihar",...,False,False,False,False,True,False,False,True,False,False
3,57,28,22,22,19-01-2025,12:27:02,14000.72,58177.08,POS Mobile App,"Chennai, Tamil Nadu",...,False,False,False,True,False,False,False,False,True,False
4,43,25,7,7,30-01-2025,18:30:46,18335.16,16108.56,Virtual Card,"Amritsar, Punjab",...,True,False,False,True,False,False,False,False,True,False


# 3. Transformar fechas y tiempos
Lo siguiente será transformar los datos fechas y horas en características numéricas.

In [58]:
df["Transaction_Date"] = pd.to_datetime(df["Transaction_Date"])
df["Transaction_Time"] = pd.to_datetime(df["Transaction_Time"], format="%H:%M:%S")

# Extraer información útil
df["Year"] = df["Transaction_Date"].dt.year
df["Month"] = df["Transaction_Date"].dt.month
df["Day"] = df["Transaction_Date"].dt.day
df["DayOfWeek"] = df["Transaction_Date"].dt.dayofweek
df["Hour"] = df["Transaction_Time"].dt.hour

# Eliminar las columnas originales
df.drop(columns=["Transaction_Date", "Transaction_Time"], inplace=True)

df.head()

C:\Users\javi2\AppData\Local\Temp\ipykernel_8828\1101040754.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["Transaction_Date"] = pd.to_datetime(df["Transaction_Date"])


,Age,State,City,Bank_Branch,Transaction_Amount,Account_Balance,Transaction_Device,Transaction_Location,Is_Fraud,Gender_Male,...,Merchant_Category_Health,Merchant_Category_Restaurant,Device_Type_Desktop,Device_Type_Mobile,Device_Type_POS,Year,Month,Day,DayOfWeek,Hour
0,60,15,127,127,32415.45,74557.27,Voice Assistant,"Thiruvananthapuram, Kerala",0,True,...,False,True,False,False,True,2025,1,23,3,16
1,51,18,100,100,43622.60,74622.66,POS Mobile Device,"Nashik, Maharashtra",0,False,...,False,True,True,False,False,2025,1,11,5,17
2,20,4,13,13,63062.56,66817.99,ATM,"Bhagalpur, Bihar",0,True,...,False,False,True,False,False,2025,1,25,5,3
3,57,28,22,22,14000.72,58177.08,POS Mobile App,"Chennai, Tamil Nadu",0,False,...,False,False,False,True,False,2025,1,19,6,12
4,43,25,7,7,18335.16,16108.56,Virtual Card,"Amritsar, Punjab",0,False,...,False,False,False,True,False,2025,1,30,3,18
